In [12]:
import tensorflow as tf


import neptune
from neptune.integrations.tensorflow_keras import NeptuneCallback

import os

import numpy as np
import pandas as pd
from window_generator import WindowGenerator

from run_automation_utils import Params

from typing import Dict

In [3]:
def prep_window_generator(station_id, second_station_id = None):
    df = pd.read_feather(f"../../data/pollution/processed/interpolated/pm10/{station_id}.feather")
    df.set_index("timestamp", inplace=True)
    
    if second_station_id is not None:
        df2 = pd.read_feather(f"../../data/pollution/processed/interpolated/pm10/{second_station_id}.feather")
        df2.set_index("timestamp", inplace=True)
        df = pd.concat([df, df2])

    timestamp_s = df.index.map(pd.Timestamp.timestamp)

    day = 24 * 60 * 60
    year = (365.2425) * day

    df["Day sin"] = np.sin(timestamp_s * (2 * np.pi / day))
    df["Day cos"] = np.cos(timestamp_s * (2 * np.pi / day))
    df["Year sin"] = np.sin(timestamp_s * (2 * np.pi / year))
    df["Year cos"] = np.cos(timestamp_s * (2 * np.pi / year))
    df.reset_index(inplace=True)

    # Convert to radians.
    wd_rad = df.pop("winddirection_10m") * np.pi / 180
    df["winddirection_10m_sin"] = np.sin(wd_rad)
    df["winddirection_10m_cos"] = np.cos(wd_rad)

    return WindowGenerator(input_width=24 * 7, label_width=24, shift=24, df=df.copy(), label_columns=["pm10"])


In [4]:
window_generator_538 = prep_window_generator(538)

In [5]:
run_params: Dict[str, Params] = {}
run_params["station_538_without_lambda"] = Params(
    window_generator_538, lstm_layer_count=2, lstm_sizes=[64, 64]
)

In [6]:
import logging

logger = logging.getLogger("run_logger")
fileHandler = logging.FileHandler("run.logs.txt")
logger.addHandler(fileHandler)

In [27]:
for run_id, params in run_params.items():
    logger.error("Start run: " + run_id)
    model = init_model(params)

    loss = tf.keras.losses.MeanSquaredError()
    if params.loss_function == "huber":
        loss = tf.keras.losses.huber
    model.compile(loss=loss, optimizer="Adam", metrics=[tf.keras.metrics.MeanAbsoluteError()])

    name = params.to_name(100)
    temp_name = name
    append_index = 0
    while os.path.exists("../models/" + temp_name):
        append_index += 1
        temp_name = name + "_" + str(append_index)
    if append_index != 0:
        name += "_" + str(append_index)

    run = neptune.init_run(
        project="data-mining-team2/model-tests",
        api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiI4Mzg2ZWZmYi05YzRlLTQ3ODYtOWE1NC1mNDM4OTM1ZjNlOTkifQ==",
        custom_run_id=run_id,
        name=name,
        source_files=["./model_tests_neighboring_stations.ipynb"],
    )

    run["parameters"] = params.to_dict()

    neptune_cbk = NeptuneCallback(run=run, base_namespace="training")

    try:
        for epoch_count_factor in range(1, 5):
            epoch_count = 50 * epoch_count_factor
            history = model.fit(
                params.window_generator.train,
                epochs=50,
                validation_data=params.window_generator.val,
                callbacks=[neptune_cbk],
            )

            eval_metrics = model.evaluate(params.window_generator.test, verbose=0)
            for j, metric in enumerate(eval_metrics):
                run[f"eval/epoch_{epoch_count}/{model.metrics_names[j]}"] = metric

            name = params.to_name(epoch_count)
            temp_name = name
            append_index = 0
            while os.path.exists("../models/" + temp_name):
                append_index += 1
                temp_name = name + "_" + str(append_index)
            if append_index != 0:
                name += "_" + str(append_index)

            model.save("../models/" + name, include_optimizer=False)

            run["model_names/" + str(epoch_count) + "epochs"] = name
    except Exception as e:
        logger.error(e)

        pass
    run.stop()
    logger.error("Finished run: " + run_id)


https://app.neptune.ai/data-mining-team2/model-tests/e/MOD-9
Epoch 1/50
682/682 [==============================] - 50s 67ms/step - loss: 243.7776 - mean_absolute_error: 11.4716 - val_loss: 222.5302 - val_mean_absolute_error: 11.1024
Epoch 2/50
682/682 [==============================] - 45s 66ms/step - loss: 223.1441 - mean_absolute_error: 10.9824 - val_loss: 222.4683 - val_mean_absolute_error: 11.0867
Epoch 3/50
682/682 [==============================] - 43s 63ms/step - loss: 223.5025 - mean_absolute_error: 10.9975 - val_loss: 222.4228 - val_mean_absolute_error: 11.0786
Epoch 4/50
682/682 [==============================] - 42s 61ms/step - loss: 223.5857 - mean_absolute_error: 10.9934 - val_loss: 222.4204 - val_mean_absolute_error: 11.0769
Epoch 5/50
682/682 [==============================] - 40s 58ms/step - loss: 223.3190 - mean_absolute_error: 10.9872 - val_loss: 222.4519 - val_mean_absolute_error: 11.0871
Epoch 6/50
682/682 [==============================] - 41s 60ms/step - loss: 223

KeyboardInterrupt: 

In [25]:
model = init_model(run_params["station_538_without_lambda"])
model.compile(loss=tf.keras.losses.MeanSquaredError(), optimizer="Adam", metrics=[tf.keras.metrics.MeanAbsoluteError()])

In [26]:
history = model.fit(
    window_generator_538.train,
    epochs=50,
    validation_data=window_generator_538.val
)

Epoch 1/50
682/682 [==============================] - ETA: 0s - loss: 244.2702 - mean_absolute_error: 11.4811

KeyboardInterrupt: 

In [23]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization_4 (Normalizat  (None, 168, 11)          23        
 ion)                                                            
                                                                 
 lambda_4 (Lambda)           (None, 168, 11)           0         
                                                                 
 conv1d_12 (Conv1D)          (None, 168, 128)          7168      
                                                                 
 max_pooling1d_8 (MaxPooling  (None, 84, 128)          0         
 1D)                                                             
                                                                 
 batch_normalization_8 (Batc  (None, 84, 128)          512       
 hNormalization)                                                 
                                                      

In [24]:
def init_model(params: Params):
    num_features = 1
    init_kernel_size = params.kernel_sizes[0]
    cnn_lstm_model = tf.keras.models.Sequential()
    # Shape [batch, time, features] => [batch, CONV_WIDTH, features]
    cnn_lstm_model.add(tf.keras.layers.Normalization())
    cnn_lstm_model.add(tf.keras.layers.Lambda(lambda x: x[:, :, :]))
    # Shape => [batch, 1, conv_units]
    cnn_lstm_model.add(
        tf.keras.layers.Conv1D(
            params.conv_layer_sizes[0], activation="relu", kernel_size=params.kernel_sizes[0], padding="same"
        )
    )
    if params.pooling_type == "MaxPooling":
        cnn_lstm_model.add(tf.keras.layers.MaxPooling1D())
    else:
        cnn_lstm_model.add(tf.keras.layers.AveragePooling1D())
    cnn_lstm_model.add(tf.keras.layers.BatchNormalization())  # TODO: useful to put it here?
    cnn_lstm_model.add(
        tf.keras.layers.Conv1D(
            params.conv_layer_sizes[1], activation="relu", kernel_size=params.kernel_sizes[1], padding="same"
        )
    )
    if params.pooling_type == "MaxPooling":
        cnn_lstm_model.add(tf.keras.layers.MaxPooling1D())
    else:
        cnn_lstm_model.add(tf.keras.layers.AveragePooling1D())
    cnn_lstm_model.add(tf.keras.layers.BatchNormalization())  # TODO: useful to put it here?
    cnn_lstm_model.add(
        tf.keras.layers.Conv1D(
            params.conv_layer_sizes[2], activation="relu", kernel_size=params.kernel_sizes[2], padding="same"
        )
    )

    cnn_lstm_model.add(tf.keras.layers.Reshape((-1, 256 * 42)))
    
    # cnn_lstm_model.add(tf.keras.layers.Flatten())

    if params.lstm_layer_count != len(params.lstm_sizes):
        raise ValueError("lstm_layer_count must be same as the length of lstm_sizes!")

    # LSTM
    for i in range(params.lstm_layer_count):
        if i == params.lstm_layer_count:
            lstm_layer2 = tf.keras.layers.LSTM(params.lstm_sizes[i], return_sequences=params.lstm_return_sequences)
            if params.lstm_bidirectional:
                cnn_lstm_model.add(tf.keras.layers.Bidirectional(lstm_layer2))
            else:
                cnn_lstm_model.add(lstm_layer2)
        else:
            lstm_layer1 = tf.keras.layers.LSTM(params.lstm_sizes[i], return_sequences=True)
            if params.lstm_bidirectional:
                cnn_lstm_model.add(tf.keras.layers.Bidirectional(lstm_layer1))
            else:
                cnn_lstm_model.add(lstm_layer1)

    for i in range(params.dense_layer_count):
        cnn_lstm_model.add(tf.keras.layers.Dense(params.dense_layer_sizes[i], activation="relu"))
        cnn_lstm_model.add(tf.keras.layers.Dropout(params.dropout))

    cnn_lstm_model.add(
        tf.keras.layers.Dense(params.label_width * num_features, kernel_initializer=tf.initializers.zeros())
    )
    # Shape => [batch, out_steps, features] -> for each prediction step one neuron
    cnn_lstm_model.add(tf.keras.layers.Reshape([params.label_width, num_features]))
    return cnn_lstm_model
